<a href="https://colab.research.google.com/github/vsnupoudel/data-science-capstone-JohnsHopkins/blob/master/Keras_for_R_for_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## After setting working directory

In [1]:
getwd()

[1] "/content"

In [3]:
install.packages( c( "tm","ngram","dplyr","tidyr","keras","data.table","stringr") )
x <- c( "tm", "ngram", "dplyr", "tidyr", "keras", "data.table")
z = lapply(x, require, character.only = TRUE)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘png’, ‘config’, ‘NLP’, ‘slam’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’


Loading required package: tm

Loading required package: NLP

Loading required package: ngram

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

Loading required package: keras

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [4]:
system("gdown --id 1hlvOXJqe3fdxjQilNfLRwPud_KWcOVEE -O '/content/en_US.zip' ")

In [5]:
unzip(zipfile= '/content/en_US.zip', overwrite = TRUE )

In [6]:
system(" gdown --id 1r3MHZdvnlC_BG8z6Jr1td-zJTC2GdqYK -O 'glove.6B.50d.txt'  ")

In [7]:
# # establish connections with three files 
print( getwd())
con3 <- file( '/content/en_US/en_US.twitter.txt', 'r')
text3 <- readLines(con3)
close(con3)

[1] "/content"


Warning message in readLines(con3):
“line 167155 appears to contain an embedded nul”
Warning message in readLines(con3):
“line 268547 appears to contain an embedded nul”
Warning message in readLines(con3):
“line 1274086 appears to contain an embedded nul”
Warning message in readLines(con3):
“line 1759032 appears to contain an embedded nul”


In [8]:
# # sample 0.01% data
sample_index <- sample(seq(text3), 0.01*length(text3))
sample_text <- text3[sample_index]

dir.create(file.path("./sample_text"))
con4 <- file('./sample_text/sampled_text.txt')
writeLines(sample_text, con = con4)
close(con4)

In [9]:
length( sample_text )

[1] 23601

In [10]:
# Using the tm library for preprocessing the sample text
library(tm)
data_folder = DirSource(directory = "./sample_text",
            encoding = "UTF-8",
            mode = "text")
# load the file into Corpus object
docs <- SimpleCorpus( data_folder )
# Converting to lowercase:
docs <- tm_map(docs, tolower)
docs <- tm_map(docs, PlainTextDocument)

Warning message in tm_map.SimpleCorpus(docs, PlainTextDocument):
“transformation drops documents”


In [11]:
length(docs)

[1] 2

In [12]:
library(ngram)
string <- docs[[1]]$content
# trying different libraries
# library(tokenizers)
# library(word2vec)
# string.summary(string)
ng <- ngram (string, n=3)
# get.phrasetable(ng)
# tokens <- tokenize_ngrams(string , n = 3 )

# As a data frame
df <- data.frame(get.phrasetable(ng))

# Put the last word as target
library(tidyr)
sep<- separate(data = df, col = ngrams , into = c("X1","X2", "y")
               # ,sep = "^\\w+\\s+"
               # , remove=FALSE
               )
# Xy <- unite(sep , X , c(X1, X2 ), remove=TRUE, sep=" ")
df <- data.frame(sep)
remove(sep)
head(df)

Warning message:
“Expected 3 pieces. Additional pieces discarded in 255140 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 99 rows [738, 1404, 2856, 7110, 7567, 9328, 14572, 22478, 24669, 25197, 30082, 31148, 31341, 33995, 35801, 36296, 36568, 42245, 42293, 46490, ...].”


,X1,X2,y,freq,prop
,<chr>,<chr>,<chr>,<int>,<dbl>
1,going,to,be,72,0.0002570795
2,a,lot,of,57,0.0002035213
3,to,be,a,56,0.0001999507
4,is,going,to,55,0.0001963802
5,i,want,to,55,0.0001963802
6,can,t,wait,51,0.0001820980


In [13]:
dim(df)

[1] 260669      5

References:

[Keras Article](https://keras.rstudio.com/articles/examples/pretrained_word_embeddings.html)

In [14]:
# Try downloading pre-trained word embeddings instead of training new
# This example shows how one can quickly load glove vectors
# and train a Keras model in R
library(keras)
library(dplyr)

# Load the word embeddings as df 
# load glove vectors into R
vectors = data.table::fread('glove.6B.50d.txt', data.table = F
                            ,  encoding = 'UTF-8')
colnames(vectors) = c('word',paste('dim',1:50,sep = '_'))

vecDf <- as_tibble(vectors)
head(vecDf)

Warning message in data.table::fread("glove.6B.50d.txt", data.table = F, encoding = "UTF-8"):
“Found and resolved improper quoting in first 100 rows. If the fields are not quoted (e.g. field separator does not appear within any field), try quote="" to avoid this warning.”


word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,⋯,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
the,0.418000,0.249680,-0.41242,0.121700,0.34527,-0.044457,-0.49688,-0.17862,-0.00066023,⋯,-2.9871e-01,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.0027849,-0.184110,-0.115140,-0.78581
",",0.013441,0.236820,-0.16899,0.409510,0.63812,0.477090,-0.42852,-0.55641,-0.36400000,⋯,-8.0262e-02,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.3781800,-0.566570,0.044691,0.30392
.,0.151640,0.301770,-0.16763,0.176840,0.31719,0.339730,-0.43478,-0.31086,-0.44999000,⋯,-6.3681e-05,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.0808030,-0.356520,0.016413,0.10216
of,0.708530,0.570880,-0.47160,0.180480,0.54449,0.726030,0.18157,-0.52393,0.10381000,⋯,-3.4727e-01,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.2161700,-0.225620,-0.093918,-0.80375
to,0.680470,-0.039263,0.30186,-0.177920,0.42962,0.032246,-0.41376,0.13228,-0.29847000,⋯,-9.4375e-02,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.0943400,-0.073297,-0.064699,-0.26044
and,0.268180,0.143460,-0.27877,0.016257,0.11384,0.699230,-0.51332,-0.47368,-0.33075000,⋯,-6.9043e-02,0.368850,0.251680,-0.245170,0.25381,0.136700,-0.3117800,-0.632100,-0.250280,-0.38097


In [15]:
# Convert the ngrams df to vectors 
# define parameters of Keras model
library(keras)
max_words = 1e4
maxlen = 2
dim_size = 50
# tokenize the input data and then fit the created object
X_seg = text_tokenizer(num_words = max_words ) %>%
  fit_text_tokenizer( paste( c(df$X1, df$X2 , df$y ) , sep=" ") )
X_seg

<keras_preprocessing.text.Tokenizer>

In [16]:
# apply tokenizer to the text and get indices instead of words
# later pad the sequence
Xtrain = texts_to_sequences(X_seg , paste( df$X1, df$X2  , sep=" ") )  %>%
 pad_sequences( maxlen = maxlen)
tail(Xtrain,3)

"[260667,]",94,76
"[260668,]",2835,284
"[260669,]",232,1754


In [17]:
#tokenize Ytrain as well
# tokenize the input data and then fit the created object
# Y_seg = text_tokenizer(num_words = max_words ) %>%
#   fit_text_tokenizer( df$y )
# Y_seg
# apply tokenizer to the text and get indices instead of words
# later pad the sequence
Ytrain = texts_to_sequences(X_seg , df$y ) %>%
 pad_sequences( maxlen = 1)
 head(Ytrain,3)

22
10
4


In [18]:
# Unlist and make dataframe  and join with 
# downloaded pretrained word Embeddings
# unlist word indices
word_indices = unlist( X_seg$word_index)
# word_indices_Y = unlist(Y_seg$word_index)
tail(word_indices,3)

arlo  0888 dangg 
24614 24615 24616

In [19]:
# then place them into data.frame 
dic = data.frame(word = names(word_indices) , key = word_indices, stringsAsFactors = FALSE) %>%
  arrange(key) %>% .[1:max_words,]
dic <- data.frame( word = unique(dic$word))
tail(dic)

,word
,<chr>
9995,essence
9996,anyday
9997,tiffany
9998,conan
9999,swingin
10000,wa


In [20]:
# join the words with GloVe vectors and
# if word does not exist in GloVe, then fill NA's with 0
word_embeds = dic %>% left_join(vectors) %>% .[,2:51] %>% replace(., is.na(.), 0) %>% as.matrix()


Joining, by = "word"



In [21]:
dim(word_embeds)

[1] 10000    50

In [22]:
# install.packages("sjlabelled")
# library(sjlabelled)
# Ytrain <- as_factor(Ytrain)

In [29]:
# Use Keras Functional API 
input = layer_input(shape = list(maxlen), name = "input")

model = input %>%
  layer_embedding(input_dim = max_words, output_dim = dim_size, input_length = maxlen,
                  # put weights into list and do not allow training
                  weights = list(word_embeds), trainable = FALSE) %>%
  
  layer_lstm(units = 10 , return_sequences = TRUE )%>%
  layer_lstm(units = 5  )

  lastLayer = model %>% layer_dense(units= max_words, activation='softmax')
  
output =   lastLayer

model = keras_model(input, output)

# Compile and fit
# instead of accuracy we can use "AUC" metrics from "tensorflow.keras"
model %>% compile(
  optimizer = "adam",
  loss = "sparse_categorical_crossentropy",
  metrics = c('accuracy')
)

In [30]:
summary(model)

Model: "model_1"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input (InputLayer)                  [(None, 2)]                     0           
________________________________________________________________________________
embedding_1 (Embedding)             (None, 2, 50)                   500000      
________________________________________________________________________________
lstm_3 (LSTM)                       (None, 2, 10)                   2440        
________________________________________________________________________________
lstm_2 (LSTM)                       (None, 5)                       320         
________________________________________________________________________________
dense_1 (Dense)                     (None, 10000)                   60000       
Total params: 562,760
Trainable params: 62,760
Non-trainable params: 500,000
_______________

In [ ]:
fit(model,
  Xtrain,
   Ytrain,
  epochs = 55,
  verbose = 1
)

In [ ]:
evaluate(model,
  Xtrain,
   Ytrain)

Prediction process

In [ ]:
predict(model, Xtrain[1, : ])

In [ ]:
Xtrain[1, ]

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
encoded_text = texts_to_sequences(X_seg , paste( "good", "good"  , sep=" ")  )
pad_encoded = pad_sequences(c(encoded_text), maxlen=2, truncating='pre')
pad_encoded

In [ ]:
predict(model,pad_encoded)

In [ ]:
# install.packages('ramify')
library(ramify)

In [ ]:
i = argmax ( predict(model,pad_encoded) )
i

In [ ]:
# for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
pred_word =dic[1,]
pred_word
# print("Next word suggestion:",pred_word)